In [1]:
from collections import Counter
from operator import itemgetter

# Burrows-Wheeler

In [2]:
def permutations(txt):
    ans = [txt]
    i = 1
    while i < len(txt):
        ans += [ans[-1][-1] + ans[-1][0:-1]]
        i += 1
    return ans

def encode_burrows_wheeler(txt):
    perms = sorted(permutations(txt))
    last_col = ''.join([x[-1] for x in perms])
    pos = perms.index(txt)
    return last_col, pos

In [3]:
def decode_burrows_wheeler(last_col, pos):
    n = len(last_col)
    first_col = sorted([(i, x) for i, x in enumerate(last_col)], key=itemgetter(1))
    T = [None for i in range(n)]
    for i, y in enumerate(first_col):
        j, _ = y
        T[j] = i
    Tx = [pos]
    for i in range(1, n):
        Tx.append(T[Tx[i - 1]])
    S = [last_col[i] for i in Tx]
    S.reverse()
    return ''.join(S)

# Move to front

In [4]:
def encode_move_to_front(txt):
    alphabet = list(set(txt))
    alphabet.sort()
    q = len(alphabet)
    sequence = []
    for char in txt:
        index = alphabet.index(char)
        sequence.append(index)
        alphabet.pop(index)
        alphabet.insert(0, char)
    return sequence

In [5]:
def decode_move_to_front(cod, alp):
    alphabet = list(alp)
    txt = ""
    for index in cod:
        char = alphabet[index]
        txt += char
        alphabet.pop(index)
        alphabet.insert(0, char)
    return txt

# Huffman

### Source from text

In [6]:
def source_fromtext(txt, n=1):
    freq_packs = {}
    for i in range(len(txt) - n + 1):
        packs = txt[i:i+n]
        if packs in freq_packs:
            freq_packs[packs] += 1
        else:
            freq_packs[packs] = 1
    freq_list = [(k, v) for k, v in freq_packs.items()]
    return sorted(freq_list, key=lambda x: x[0])

### Canonical code

In [7]:
def kraft_inequality(lengths, q):
    s = 0
    for l in lengths:
        s += q**-l

    return s <= 1

def format_to_alf(number, base, length, alf):
    if number == 0:
        res = alf[0]
        if length == 1:
            return res
        else:
            count = 1
            digits = []
            digits.append(res)
            while (count < length):
                digits.insert(0, alf[0])
                count +=1
            res = ''.join(str(e) for e in digits[::-1])
            return res

    digits = []
    while number > 0:
        digits.append(alf[int(number % base)])
        number //= base
    count = len(digits)
    while (count < length):
                digits.append(alf[0])
                count +=1        
    res = ''.join(str(e) for e in digits[::-1])
    return res

In [8]:
def canonical_code(L,q=2, alf = [0,1]):
    if not kraft_inequality(L, q):
        return 'The entry does not satisfy Kraft-McMillan inequality.'
    
    bl_count = Counter(L)
    code = 0
    bl_count[0] = 0
    next_code = {}
    maximum = max(L) + 1       
    for l in range (1, maximum):
        code = (code + bl_count[l-1])*q
        next_code[l] = code 
    def_code = []
    lengths = {}
    for l in L:
        length = l
        def_code.append(next_code[length])
        lengths[next_code[length]] = length
        next_code[length] += 1
    def_code = list(map(lambda x: format_to_alf(x,q,lengths[x], alf),def_code))
    return def_code

### Huffman

In [9]:
def huffman_code(txt, src, package_size=1):
    #src = source_fromtext(txt,package_size)
    d_nodes = {}
    for c in src:
        d_nodes[c[0]] = 0
    
    sorted_d = sorted(src, key=lambda x: x[1]) 

    while len(sorted_d) > 1:
        new_c = sorted_d[0][0] + sorted_d[1][0]
        new_f = sorted_d[0][1] + sorted_d[1][1]
        
        for i in range(0, len(sorted_d[0][0]), package_size):
            package = sorted_d[0][0][i:i+package_size]
            d_nodes[package] += 1
            
        for i in range(0, len(sorted_d[1][0]), package_size):
            package = sorted_d[1][0][i:i+package_size]
            d_nodes[package] += 1
        
        sorted_d[1] = (new_c,new_f)
        sorted_d.pop(0);
        sorted_d = sorted(sorted_d, key=lambda x: x[1])
    
    result = [(key,value) for key, value in zip(d_nodes.keys(), canonical_code(d_nodes.values(), 2, ['0','1']))]
    return result

# Encode

In [10]:
def encode(txta,corr):
    corr = dict(corr)
    txt_encoded = ''
    i, j = 0, 0
    while j<=len(txta):
        substring = txta[i:j]
        if substring in corr:
            txt_encoded += corr[substring]
            i = j
        j += 1
        
    if i != len(txta): # all the text could not be processed
        return 'Message could not be encoded'
    return txt_encoded

# Decode

In [11]:
def decode(txtb,corr):
    corr = dict(corr)
    corr_keys = list(corr.keys())
    corr_values = list(corr.values())
    txt_decoded = ''
    i, j = 0, 0
    while j<=len(txtb):
        substring = txtb[i:j]
        if substring in corr_values:
            pos = corr_values.index(substring)
            txt_decoded += corr_keys[pos]
            i = j
        j += 1
        
    if i != len(txtb): # all the text could not be processed
        return 'Message could not be decoded'
    return txt_decoded

# Testing

In [12]:
txt1 = "En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza. Quieren decir que tenía el sobrenombre de Quijada, o Quesada, que en esto hay alguna diferencia en los autores que deste caso escriben; aunque, por conjeturas verosímiles, se deja entender que se llamaba Quejana. Pero esto importa poco a nuestro cuento; basta que en la narración dél no se salga un punto de la verdad.\nEs, pues, de saber que este sobredicho hidalgo, los ratos que estaba ocioso, que eran los más del año, se daba a leer libros de caballerías, con tanta afición y gusto, que olvidó casi de todo punto el ejercicio de la caza, y aun la administración de su hacienda. Y llegó a tanto su curiosidad y desatino en esto, que vendió muchas hanegas de tierra de sembradura para comprar libros de caballerías en que leer, y así, llevó a su casa todos cuantos pudo haber dellos; y de todos, ningunos le parecían tan bien como los que compuso el famoso Feliciano de Silva, porque la claridad de su prosa y aquellas entricadas razones suyas le parecían de perlas, y más cuando llegaba a leer aquellos requiebros y cartas de desafíos, donde en muchas partes hallaba escrito: La razón de la sinrazón que a mi razón se hace, de tal manera mi razón enflaquece, que con razón me quejo de la vuestra fermosura. Y también cuando leía: ...los altos cielos que de vuestra divinidad divinamente con las estrellas os fortifican, y os hacen merecedora del merecimiento que merece la vuestra grandeza.\nCon estas razones perdía el pobre caballero el juicio, y desvelábase por entenderlas y desentrañarles el sentido, que no se lo sacara ni las entendiera el mesmo Aristóteles, si resucitara para sólo ello. No estaba muy bien con las heridas que don Belianís daba y recebía, porque se imaginaba que, por grandes maestros que le hubiesen curado, no dejaría de tener el rostro y todo el cuerpo lleno de cicatrices y señales. Pero, con todo, alababa en su autor aquel acabar su libro con la promesa de aquella inacabable aventura, y muchas veces le vino deseo de tomar la pluma y dalle fin al pie de la letra, como allí se promete; y sin duda alguna lo hiciera, y aun saliera con ello, si otros mayores y continuos pensamientos no se lo estorbaran. Tuvo muchas veces competencia con el cura de su lugar -que era hombre docto, graduado en Sigüenza-, sobre cuál había sido mejor caballero: Palmerín de Ingalaterra o Amadís de Gaula; mas maese Nicolás, barbero del mesmo pueblo, decía que ninguno llegaba al Caballero del Febo, y que si alguno se le podía comparar, era don Galaor, hermano de Amadís de Gaula, porque tenía muy acomodada condición para todo; que no era caballero melindroso, ni tan llorón como su hermano, y que en lo de la valentía no le iba en zaga.\nEn resolución, él se enfrascó tanto en su letura, que se le pasaban las noches leyendo de claro en claro, y los días de turbio en turbio; y así, del poco dormir y del mucho leer, se le secó el celebro, de manera que vino a perder el juicio. Llenósele la fantasía de todo aquello que leía en los libros, así de encantamentos como de pendencias, batallas, desafíos, heridas, requiebros, amores, tormentas y disparates imposibles; y asentósele de tal modo en la imaginación que era verdad toda aquella máquina de aquellas sonadas soñadas invenciones que leía, que para él no había otra historia más cierta en el mundo. Decía él que el Cid Ruy Díaz había sido muy buen caballero, pero que no tenía que ver con el Caballero de la Ardiente Espada, que de sólo un revés había partido por medio dos fieros y descomunales gigantes. Mejor estaba con Bernardo del Carpio, porque en Roncesvalles había muerto a Roldán el encantado, valiéndose de la industria de Hércules, cuando ahogó a Anteo, el hijo de la Tierra, entre los brazos. Decía mucho bien del gigante Morgante, porque, con ser de aquella generación gigantea, que todos son soberbios y descomedidos, él solo era afable y bien criado. Pero, sobre todos, estaba bien con Reinaldos de Montalbán, y más cuando le veía salir de su castillo y robar cuantos topaba, y cuando en allende robó aquel ídolo de Mahoma que era todo de oro, según dice su historia. Diera él, por dar una mano de coces al traidor de Galalón, al ama que tenía, y aun a su sobrina de añadidura.\nEn efeto, rematado ya su juicio, vino a dar en el más estraño pensamiento que jamás dio loco en el mundo; y fue que le pareció convenible y necesario, así para el aumento de su honra como para el servicio de su república, hacerse caballero andante, y irse por todo el mundo con sus armas y caballo a buscar las aventuras y a ejercitarse en todo aquello que él había leído que los caballeros andantes se ejercitaban, deshaciendo todo género de agravio, y poniéndose en ocasiones y peligros donde, acabándolos, cobrase eterno nombre y fama. Imaginábase el pobre ya coronado por el valor de su brazo, por lo menos, del imperio de Trapisonda; y así, con estos tan agradables pensamientos, llevado del estraño gusto que en ellos sentía, se dio priesa a poner en efeto lo que deseaba.\nY lo primero que hizo fue limpiar unas armas que habían sido de sus bisabuelos, que, tomadas de orín y llenas de moho, luengos siglos había que estaban puestas y olvidadas en un rincón. Limpiólas y aderezólas lo mejor que pudo, pero vio que tenían una gran falta, y era que no tenían celada de encaje, sino morrión simple; mas a esto suplió su industria, porque de cartones hizo un modo de media celada, que, encajada con el morrión, hacían una apariencia de celada entera. Es verdad que para probar si era fuerte y podía estar al riesgo de una cuchillada, sacó su espada y le dio dos golpes, y con el primero y en un punto deshizo lo que había hecho en una semana; y no dejó de parecerle mal la facilidad con que la había hecho pedazos, y, por asegurarse deste peligro, la tornó a hacer de nuevo, poniéndole unas barras de hierro por de dentro, de tal manera que él quedó satisfecho de su fortaleza; y, sin querer hacer nueva experiencia della, la diputó y tuvo por celada finísima de encaje.\nFue luego a ver su rocín, y, aunque tenía más cuartos que un real y más tachas que el caballo de Gonela, que tantum pellis et ossa fuit, le pareció que ni el Bucéfalo de Alejandro ni Babieca el del Cid con él se igualaban. Cuatro días se le pasaron en imaginar qué nombre le pondría; porque, según se decía él a sí mesmo, no era razón que caballo de caballero tan famoso, y tan bueno él por sí, estuviese sin nombre conocido; y ansí, procuraba acomodársele de manera que declarase quién había sido, antes que fuese de caballero andante, y lo que era entonces; pues estaba muy puesto en razón que, mudando su señor estado, mudase él también el nombre, y le cobrase famoso y de estruendo, como convenía a la nueva orden y al nuevo ejercicio que ya profesaba. Y así, después de muchos nombres que formó, borró y quitó, añadió, deshizo y tornó a hacer en su memoria e imaginación, al fin le vino a llamar Rocinante: nombre, a su parecer, alto, sonoro y significativo de lo que había sido cuando fue rocín, antes de lo que ahora era, que era antes y primero de todos los rocines del mundo.\nPuesto nombre, y tan a su gusto, a su caballo, quiso ponérsele a sí mismo, y en este pensamiento duró otros ocho días, y al cabo se vino a llamar don Quijote; de donde -como queda dicho- tomaron ocasión los autores desta tan verdadera historia que, sin duda, se debía de llamar Quijada, y no Quesada, como otros quisieron decir. Pero, acordándose que el valeroso Amadís no sólo se había contentado con llamarse Amadís a secas, sino que añadió el nombre de su reino y patria, por Hepila famosa, y se llamó Amadís de Gaula, así quiso, como buen caballero, añadir al suyo el nombre de la suya y llamarse don Quijote de la Mancha, con que, a su parecer, declaraba muy al vivo su linaje y patria, y la honraba con tomar el sobrenombre della.\nLimpias, pues, sus armas, hecho del morrión celada, puesto nombre a su rocín y confirmándose a sí mismo, se dio a entender que no le faltaba otra cosa sino buscar una dama de quien enamorarse; porque el caballero andante sin amores era árbol sin hojas y sin fruto y cuerpo sin alma. Decíase él a sí:\n-Si yo, por malos de mis pecados, o por mi buena suerte, me encuentro por ahí con algún gigante, como de ordinario les acontece a los caballeros andantes, y le derribo de un encuentro, o le parto por mitad del cuerpo, o, finalmente, le venzo y le rindo, ¿no será bien tener a quien enviarle presentado y que entre y se hinque de rodillas ante mi dulce señora, y diga con voz humilde y rendido: ''Yo, señora, soy el gigante Caraculiambro, señor de la ínsula Malindrania, a quien venció en singular batalla el jamás como se debe alabado caballero don Quijote de la Mancha, el cual me mandó que me presentase ante vuestra merced, para que la vuestra grandeza disponga de mí a su talante''?\n¡Oh, cómo se holgó nuestro buen caballero cuando hubo hecho este discurso, y más cuando halló a quien dar nombre de su dama! Y fue, a lo que se cree, que en un lugar cerca del suyo había una moza labradora de muy buen parecer, de quien él un tiempo anduvo enamorado, aunque, según se entiende, ella jamás lo supo, ni le dio cata dello. Llamábase Aldonza Lorenzo, y a ésta le pareció ser bien darle título de señora de sus pensamientos; y, buscándole nombre que no desdijese mucho del suyo, y que tirase y se encaminase al de princesa y gran señora, vino a llamarla Dulcinea del Toboso, porque era natural del Toboso; nombre, a su parecer, músico y peregrino y significativo, como todos los demás que a él y a sus cosas había puesto."

In [13]:
txt2 = "el ingenioso hidalgo don quijote de la mancha tasa yo juan gallo de andrada escribano de camara del rey nuestro senor de los que residen en su consejo certifico y doy fe que habiendo visto por los senores del un libro intitulado el ingenioso hidalgo de la mancha compuesto por miguel de cervantes saavedra tasaron cada pliego del dicho libro a tres maravedis y medio el cual tiene ochenta y tres pliegos que al dicho precio monta el dicho libro docientos y noventa maravedis y medio en que se ha de vender en papel y dieron licencia para que a este precio se pueda vender y mandaron que esta tasa se ponga al principio del dicho libro y no se pueda vender sin ella y para que dello conste di la presente en valladolid a veinte dias del mes de deciembre de mil y seiscientos y cuatro anos juan gallo de andrada testimonio de las erratas este libro no tiene cosa digna que no corresponda a su original en testimonio de lo haber correcto di esta fee en el colegio de la madre de dios de los teologos de"

In [28]:
txt3 = "el ingenioso hidalgo don quijote de la mancha tasa yo juan gallo de andrada escribano de camara del rey nuestro senor de los que residen en su consejo certifico y doy fe que habiendo visto por los senores del un libro intitulado el ingenioso hidalgo de la mancha compuesto por miguel de cervantes saavedra tasaron cada pliego del dicho libro a tres maravedis y medio el cual tiene ochenta y tres pliegos que al dicho precio monta el dicho libro docientos y noventa maravedis y medio en que se ha de vender en papel y dieron licencia para que a este precio se pueda vender y mandaron que esta tasa se ponga al principio del dicho libro y no se pueda vender sin ella y para que dello conste di la presente en valladolid a veinte dias del mes de deciembre de mil y seiscientos y cuatro anos juan gallo de andrada testimonio de las erratas este libro no tiene cosa digna que no corresponda a su original en testimonio de lo haber correcto di esta fee en el colegio de la madre de dios de los teologos de la universidad de alcala en primero de diciembre de anos el licenciado francisco murcia de la llana el rey por cuanto por parte de vos miguel de cervantes nos fue fecha relacion que habiades compuesto un libro intitulado el ingenioso hidalgo de la mancha el cual os habia costado mucho trabajo y era muy util y provechoso nos pedistes y suplicastes os mandasemos dar licencia y facultad para le poder imprimir y previlegio por el tiempo que fuesemos servidos o como la nuestra merced fuese lo cual visto por los del nuestro consejo por cuanto en el dicho libro se hicieron las diligencias que la prematica ultimamente por nos fecha sobre la impresion de los libros dispone fue acordado que debiamos mandar dar esta nuestra cedula para vos en la dicha razon y nos tuvimoslo por bien por la cual por os hacer bien y merced os damos licencia y facultad para que vos o la persona que vuestro poder hubiere y no otra alguna podais imprimir el dicho libro intitulado el ingenioso hidalgo de la mancha que desuso se hace mencion en todos estos nuestros reinos de castilla por tiempo y espacio de diez anos que corran y se cuenten desde el dicho dia de la data desta nuestra cedula so pena que la persona o personas que sin tener vuestro poder lo imprimiere o vendiere o hiciere imprimir o vender por el mesmo caso pierda la impresion que hiciere con los moldes y aparejos della y mas incurra en pena de cincuenta mil maravedis cada vez que lo contrario hiciere la cual dicha pena sea la tercia parte para la persona que lo acusare y la otra tercia parte para nuestra camara y la otra tercia parte para el juez que lo sentenciare con tanto que todas las veces que hubieredes de hacer imprimir el dicho libro durante el tiempo de los dichos diez anos le traigais al nuestro consejo juntamente con el original que en el fue visto que va rubricado cada plana y firmado al fin del de juan gallo de andrada nuestro escribano de camara de los que en el residen para saber si la dicha impresion esta conforme el original o traigais fe en publica forma de como por corretor nombrado por nuestro mandado se vio y corrigio la dicha impresion por el original y se imprimio conforme a el y quedan impresas las erratas por el apuntadas para cada un libro de los que asi fueren impresos para que se tase el precio que por cada volume hubieredes de haber y mandamos al impresor que asi imprimiere el dicho libro no imprima el principio ni el primer pliego del ni entregue mas de un solo libro con el original al autor o persona a cuya costa lo imprimiere ni otro alguno para efeto de la dicha correcion y tasa hasta que antes y primero el dicho libro este corregido y tasado por los del nuestro consejo y estando hecho y no de otra manera pueda imprimir el dicho principio y primer pliego y sucesivamente ponga esta nuestra cedula y la aprobacion tasa y erratas so pena de caer e incurrir en las penas contenidas en las leyes y prematicas destos nuestros reinos y mandamos a los del nuestro consejo y a otras cualesquier justicias dellos guarden y cumplan esta nuestra cedula y lo en ella contenido fecha en valladolid a veinte y seis dias del mes de setiembre de mil y seiscientos y cuatro anos yo el rey por mandado del rey nuestro senor juan de amezqueta al duque de bejar marques de gibraleon conde de benalcazar y banares vizconde de la puebla de alcocer senor de las villas de capilla curiel y burguillos en fe del buen acogimiento y honra que hace vuestra excelencia a toda suerte de libros como principe tan inclinado a favorecer las buenas artes mayormente las que por su nobleza no se abaten al servicio y granjerias del vulgo he determinado de sacar a luz al ingenioso hidalgo don quijote de la mancha al abrigo del clarisimo nombre de vuestra excelencia a quien con el acatamiento que debo a tanta grandeza suplico le reciba agradablemente en su proteccion para que a su sombra aunque desnudo de aquel precioso ornamento de elegancia y erudicion de que suelen andar vestidas las obras que se componen en las casas de los hombres que saben ose parecer seguramente en el juicio de algunos que continiendose en los limites de su ignorancia suelen condenar con mas rigor y menos justicia los trabajos ajenos que poniendo los ojos la prudencia de vuestra excelencia en mi buen deseo fio que no desdenara la cortedad de tan humilde servicio miguel de cervantes saavedra prologo desocupado lector sin juramento me podras creer que quisiera que este libro como hijo del entendimiento fuera el mas hermoso el mas gallardo y mas discreto que pudiera imaginarse pero no he podido yo contravenir al orden de naturaleza que en ella cada cosa engendra su semejante y asi que podra engendrar el esteril y mal cultivado ingenio mio sino la historia de un hijo seco avellanado antojadizo y lleno de pensamientos varios y nunca imaginados de otro alguno bien como quien se engendro en una carcel donde toda incomodidad tiene su asiento y donde todo triste ruido hace su habitacion el sosiego el lugar apacible la amenidad de los campos la serenidad de los cielos el murmurar de las fuentes la quietud del espiritu son grande parte para que las musas mas esteriles se muestren fecundas y ofrezcan partos al mundo que le colmen de maravilla y de contento acontece tener un padre un hijo feo y sin gracia alguna y el amor que le tiene le pone una venda en los ojos para que no vea sus faltas antes las juzga por discreciones y lindezas y las cuenta a sus amigos por agudezas y donaires pero yo que aunque parezco padre soy padrastro de don quijote no quiero irme con la corriente del uso ni suplicarte casi con las lagrimas en los ojos como otros hacen lector carisimo que perdones o disimules las faltas que en este mi hijo vieres y ni eres su pariente ni su amigo y tienes tu alma en tu cuerpo y tu libre albedrio como el mas pintado y estas en tu casa donde eres senor della como el rey de sus alcabalas y sabes lo que comunmente se dice que debajo de mi manto al rey mato todo lo cual te esenta y hace libre de todo respecto y obligacion y asi puedes decir de la historia todo aquello que te pareciere sin temor que te calunien por el mal ni te premien por el bien que dijeres della solo quisiera dartela monda y desnuda sin el ornato de prologo ni de la inumerabilidad y catalogo de los acostumbrados sonetos epigramas y elogios que al principio de los libros suelen ponerse porque te se decir que aunque me costo algun trabajo componerla ninguno tuve por mayor que hacer esta prefacion que vas leyendo muchas veces tome la pluma para escribille y muchas la deje por no saber lo que escribiria y estando una suspenso con el papel delante la pluma en la oreja el codo en el bufete y la mano en la mejilla pensando lo que diria entro a deshora un amigo mio gracioso y bien entendido el cual viendome tan imaginativo me pregunto la causa y no encubriendosela yo le dije que pensaba en el prologo que habia de hacer a la historia de don quijote y que me tenia de suerte que ni queria hacerle ni menos sacar a luz las hazanas de tan noble caballero porque como quereis vos que no me tenga confuso el que dira el antiguo legislador que llaman vulgo cuando vea que al cabo de tantos anos como ha que duermo en el silencio del olvido salgo ahora con todos mis anos a cuestas con una leyenda seca como un esparto ajena de invencion menguada de estilo pobre de concetos y falta de toda erudicion y doctrina sin acotaciones en las margenes y sin anotaciones en el fin del libro como veo que estan otros libros aunque sean fabulosos y profanos tan llenos de sentencias de aristoteles de platon y de toda la caterva de filosofos que admiran a los leyentes y tienen a sus autores por hombres leidos eruditos y elocuentes pues que cuando citan la divina escritura no diran sino que son unos santos tomases y otros doctores de la iglesia guardando en esto un decoro tan ingenioso que en un renglon han pintado un enamorado destraido y en otro hacen un sermoncico cristiano que es un contento y un regalo oille o leelle de todo esto ha de carecer mi libro porque ni tengo que acotar en el margen ni que anotar en el fin ni menos se que autores sigo en el para ponerlos al principio como hacen todos por las letras del abc comenzando en aristoteles y acabando en xenofonte y en zoilo o zeuxis aunque fue maldiciente el uno y pintor el otro tambien ha de carecer mi libro de sonetos al principio a lo menos de sonetos cuyos autores sean duques marqueses condes obispos damas o poetas celeberrimos aunque si yo los pidiese a dos o tres oficiales amigos yo se que me los darian y tales que no les igualasen los de aquellos que tienen mas nombre en nuestra espana en fin senor y amigo mio prosegui yo determino que el senor don quijote se quede sepultado en sus archivos en la mancha hasta que el cielo depare quien le adorne de tantas cosas como le faltan porque yo me hallo incapaz de remediarlas por mi insuficiencia y pocas letras y porque naturalmente soy poltron y perezoso de andarme buscando autores que digan lo que yo me se decir sin ellos"

In [15]:
txt4 = "el ingenioso hidalgo don quijote de la mancha tasa yo juan gallo de andrada escribano de camara del rey nuestro senor de los que residen en su consejo certifico y doy fe que habiendo visto por los senores del un libro intitulado el ingenioso hidalgo de la mancha compuesto por miguel de cervantes saavedra tasaron cada pliego del dicho libro a tres maravedis y medio el cual tiene ochenta y tres pliegos que al dicho precio monta el dicho libro docientos y noventa maravedis y medio en que se ha de vender en papel y dieron licencia para que a este precio se pueda vender y mandaron que esta tasa se ponga al principio del dicho libro y no se pueda vender sin ella y para que dello conste di la presente en valladolid a veinte dias del mes de deciembre de mil y seiscientos y cuatro anos juan gallo de andrada testimonio de las erratas este libro no tiene cosa digna que no corresponda a su original en testimonio de lo haber correcto di esta fee en el colegio de la madre de dios de los teologos de la universidad de alcala en primero de diciembre de anos el licenciado francisco murcia de la llana el rey por cuanto por parte de vos miguel de cervantes nos fue fecha relacion que habiades compuesto un libro intitulado el ingenioso hidalgo de la mancha el cual os habia costado mucho trabajo y era muy util y provechoso nos pedistes y suplicastes os mandasemos dar licencia y facultad para le poder imprimir y previlegio por el tiempo que fuesemos servidos o como la nuestra merced fuese lo cual visto por los del nuestro consejo por cuanto en el dicho libro se hicieron las diligencias que la prematica ultimamente por nos fecha sobre la impresion de los libros dispone fue acordado que debiamos mandar dar esta nuestra cedula para vos en la dicha razon y nos tuvimoslo por bien por la cual por os hacer bien y merced os damos licencia y facultad para que vos o la persona que vuestro poder hubiere y no otra alguna podais imprimir el dicho libro intitulado el ingenioso hidalgo de la mancha que desuso se hace mencion en todos estos nuestros reinos de castilla por tiempo y espacio de diez anos que corran y se cuenten desde el dicho dia de la data desta nuestra cedula so pena que la persona o personas que sin tener vuestro poder lo imprimiere o vendiere o hiciere imprimir o vender por el mesmo caso pierda la impresion que hiciere con los moldes y aparejos della y mas incurra en pena de cincuenta mil maravedis cada vez que lo contrario hiciere la cual dicha pena sea la tercia parte para la persona que lo acusare y la otra tercia parte para nuestra camara y la otra tercia parte para el juez que lo sentenciare con tanto que todas las veces que hubieredes de hacer imprimir el dicho libro durante el tiempo de los dichos diez anos le traigais al nuestro consejo juntamente con el original que en el fue visto que va rubricado cada plana y firmado al fin del de juan gallo de andrada nuestro escribano de camara de los que en el residen para saber si la dicha impresion esta conforme el original o traigais fe en publica forma de como por corretor nombrado por nuestro mandado se vio y corrigio la dicha impresion por el original y se imprimio conforme a el y quedan impresas las erratas por el apuntadas para cada un libro de los que asi fueren impresos para que se tase el precio que por cada volume hubieredes de haber y mandamos al impresor que asi imprimiere el dicho libro no imprima el principio ni el primer pliego del ni entregue mas de un solo libro con el original al autor o persona a cuya costa lo imprimiere ni otro alguno para efeto de la dicha correcion y tasa hasta que antes y primero el dicho libro este corregido y tasado por los del nuestro consejo y estando hecho y no de otra manera pueda imprimir el dicho principio y primer pliego y sucesivamente ponga esta nuestra cedula y la aprobacion tasa y erratas so pena de caer e incurrir en las penas contenidas en las leyes y prematicas destos nuestros reinos y mandamos a los del nuestro consejo y a otras cualesquier justicias dellos guarden y cumplan esta nuestra cedula y lo en ella contenido fecha en valladolid a veinte y seis dias del mes de setiembre de mil y seiscientos y cuatro anos yo el rey por mandado del rey nuestro senor juan de amezqueta al duque de bejar marques de gibraleon conde de benalcazar y banares vizconde de la puebla de alcocer senor de las villas de capilla curiel y burguillos en fe del buen acogimiento y honra que hace vuestra excelencia a toda suerte de libros como principe tan inclinado a favorecer las buenas artes mayormente las que por su nobleza no se abaten al servicio y granjerias del vulgo he determinado de sacar a luz al ingenioso hidalgo don quijote de la mancha al abrigo del clarisimo nombre de vuestra excelencia a quien con el acatamiento que debo a tanta grandeza suplico le reciba agradablemente en su proteccion para que a su sombra aunque desnudo de aquel precioso ornamento de elegancia y erudicion de que suelen andar vestidas las obras que se componen en las casas de los hombres que saben ose parecer seguramente en el juicio de algunos que continiendose en los limites de su ignorancia suelen condenar con mas rigor y menos justicia los trabajos ajenos que poniendo los ojos la prudencia de vuestra excelencia en mi buen deseo fio que no desdenara la cortedad de tan humilde servicio miguel de cervantes saavedra prologo desocupado lector sin juramento me podras creer que quisiera que este libro como hijo del entendimiento fuera el mas hermoso el mas gallardo y mas discreto que pudiera imaginarse pero no he podido yo contravenir al orden de naturaleza que en ella cada cosa engendra su semejante y asi que podra engendrar el esteril y mal cultivado ingenio mio sino la historia de un hijo seco avellanado antojadizo y lleno de pensamientos varios y nunca imaginados de otro alguno bien como quien se engendro en una carcel donde toda incomodidad tiene su asiento y donde todo triste ruido hace su habitacion el sosiego el lugar apacible la amenidad de los campos la serenidad de los cielos el murmurar de las fuentes la quietud del espiritu son grande parte para que las musas mas esteriles se muestren fecundas y ofrezcan partos al mundo que le colmen de maravilla y de contento acontece tener un padre un hijo feo y sin gracia alguna y el amor que le tiene le pone una venda en los ojos para que no vea sus faltas antes las juzga por discreciones y lindezas y las cuenta a sus amigos por agudezas y donaires pero yo que aunque parezco padre soy padrastro de don quijote no quiero irme con la corriente del uso ni suplicarte casi con las lagrimas en los ojos como otros hacen lector carisimo que perdones o disimules las faltas que en este mi hijo vieres y ni eres su pariente ni su amigo y tienes tu alma en tu cuerpo y tu libre albedrio como el mas pintado y estas en tu casa donde eres senor della como el rey de sus alcabalas y sabes lo que comunmente se dice que debajo de mi manto al rey mato todo lo cual te esenta y hace libre de todo respecto y obligacion y asi puedes decir de la historia todo aquello que te pareciere sin temor que te calunien por el mal ni te premien por el bien que dijeres della solo quisiera dartela monda y desnuda sin el ornato de prologo ni de la inumerabilidad y catalogo de los acostumbrados sonetos epigramas y elogios que al principio de los libros suelen ponerse porque te se decir que aunque me costo algun trabajo componerla ninguno tuve por mayor que hacer esta prefacion que vas leyendo muchas veces tome la pluma para escribille y muchas la deje por no saber lo que escribiria y estando una suspenso con el papel delante la pluma en la oreja el codo en el bufete y la mano en la mejilla pensando lo que diria entro a deshora un amigo mio gracioso y bien entendido el cual viendome tan imaginativo me pregunto la causa y no encubriendosela yo le dije que pensaba en el prologo que habia de hacer a la historia de don quijote y que me tenia de suerte que ni queria hacerle ni menos sacar a luz las hazanas de tan noble caballero porque como quereis vos que no me tenga confuso el que dira el antiguo legislador que llaman vulgo cuando vea que al cabo de tantos anos como ha que duermo en el silencio del olvido salgo ahora con todos mis anos a cuestas con una leyenda seca como un esparto ajena de invencion menguada de estilo pobre de concetos y falta de toda erudicion y doctrina sin acotaciones en las margenes y sin anotaciones en el fin del libro como veo que estan otros libros aunque sean fabulosos y profanos tan llenos de sentencias de aristoteles de platon y de toda la caterva de filosofos que admiran a los leyentes y tienen a sus autores por hombres leidos eruditos y elocuentes pues que cuando citan la divina escritura no diran sino que son unos santos tomases y otros doctores de la iglesia guardando en esto un decoro tan ingenioso que en un renglon han pintado un enamorado destraido y en otro hacen un sermoncico cristiano que es un contento y un regalo oille o leelle de todo esto ha de carecer mi libro porque ni tengo que acotar en el margen ni que anotar en el fin ni menos se que autores sigo en el para ponerlos al principio como hacen todos por las letras del abc comenzando en aristoteles y acabando en xenofonte y en zoilo o zeuxis aunque fue maldiciente el uno y pintor el otro tambien ha de carecer mi libro de sonetos al principio a lo menos de sonetos cuyos autores sean duques marqueses condes obispos damas o poetas celeberrimos aunque si yo los pidiese a dos o tres oficiales amigos yo se que me los darian y tales que no les igualasen los de aquellos que tienen mas nombre en nuestra espana en fin senor y amigo mio prosegui yo determino que el senor don quijote se quede sepultado en sus archivos en la mancha hasta que el cielo depare quien le adorne de tantas cosas como le faltan porque yo me hallo incapaz de remediarlas por mi insuficiencia y pocas letras y porque naturalmente soy poltron y perezoso de andarme buscando autores que digan lo que yo me se decir sin ellos de aqui nace la suspension y elevamiento amigo en que me hallastes bastante causa para ponerme en ella la que de mi habeis oido oyendo lo cual mi amigo dandose una palmada en la frente y disparando en una carga de risa me dijo por dios hermano que agora me acabo de desenganar de un engano en que he estado todo el mucho tiempo que ha que os conozco en el cual siempre os he tenido por discreto y prudente en todas vuestras aciones pero agora veo que estais tan lejos de serlo como lo esta el cielo de la tierra como que es posible que cosas de tan poco momento y tan faciles de remediar puedan tener fuerzas de suspender y absortar un ingenio tan maduro como el vuestro y tan hecho a romper y atropellar por otras dificultades mayores a la fe esto no nace de falta de habilidad sino de sobra de pereza y penuria de discurso quereis ver si es verdad lo que digo pues estadme atento y vereis como en un abrir y cerrar de ojos confundo todas vuestras dificultades y remedio todas las faltas que decis que os suspenden y acobardan para dejar de sacar a la luz del mundo la historia de vuestro famoso don quijote luz y espejo de toda la caballeria andante decid le replique yo oyendo lo que me decia de que modo pensais llenar el vacio de mi temor y reducir a claridad el caos de mi confusion a lo cual el dijo lo primero en que reparais de los sonetos epigramas o elogios que os faltan para el principio y que sean de personajes graves y de titulo se puede remediar en que vos mesmo tomeis algun trabajo en hacerlos y despues los podeis bautizar y poner el nombre que quisieredes ahijandolos al preste juan de las indias o al emperador de trapisonda de quien yo se que hay noticia que fueron famosos poetas y cuando no lo hayan sido y hubiere algunos pedantes y bachilleres que por detras os muerdan y murmuren desta verdad no se os de dos maravedis porque ya que os averiguen la mentira no os han de cortar la mano con que lo escribistes en lo de citar en las margenes los libros y autores de donde sacaredes las sentencias y dichos que pusieredes en vuestra historia no hay mas sino hacer de manera que venga a pelo algunas sentencias o latines que vos sepais de memoria o a lo menos que os cuesten poco trabajo el buscalle como sera poner tratando de libertad y cautiverio non bene pro toto libertas venditur auro y luego en el margen citar a horacio o a quien lo dijo si trataredes del poder de la muerte acudir luego con pallida mors aequo pulsat pede pauperum tabernas regumque turres si de la amistad y amor que dios manda que se tenga al enemigo entraros luego al punto por la escritura divina que lo podeis hacer con tantico de curiosidad y decir las palabras por lo menos del mismo dios ego autem dico vobis diligite inimicos vestros si trataredes de malos pensamientos acudid con el evangelio de corde exeunt cogitationes malae si de la instabilidad de los amigos ahi esta caton que os dara su distico donec eris felix multos numerabis amicos tempora si fuerint nubila solus eris y con estos latinicos y otros tales os tendran siquiera por gramatico que el serlo no es de poca honra y provecho el dia de hoy en lo que toca el poner anotaciones al fin del libro seguramente lo podeis hacer desta manera si nombrais algun gigante en vuestro libro hacelde que sea el gigante golias y con solo esto que os costara casi nada teneis una grande anotacion pues podeis poner el gigante golias o goliat fue un filisteo a quien el pastor david mato de una gran pedrada en el valle de terebinto segun se cuenta en el libro de los reyes en el capitulo que vos hallaredes que se escribe tras esto para mostraros hombre erudito en letras humanas y cosmografo haced de modo como en vuestra historia se nombre el rio tajo y vereisos luego con otra famosa anotacion poniendo el rio tajo fue asi dicho por un rey de las espanas tiene su nacimiento en tal lugar y muere en el mar oceano besando los muros de la famosa ciudad de lisboa y es opinion que tiene las arenas de oro etc si trataredes de ladrones yo os dire la historia de caco que la se de coro si de mujeres rameras ahi esta el obispo de mondonedo que os prestara a lamia laida y flora cuya anotacion os dara gran credito si de crueles ovidio os entregara a medea si de encantadores y hechiceras homero tiene a calipso y virgilio a circe si de capitanes valerosos el mesmo julio cesar os prestara a si mismo en sus comentarios y plutarco os dara mil alejandros si trataredes de amores con dos onzas que sepais de la lengua toscana topareis con leon hebreo que os hincha las medidas y si no quereis andaros por tierras extranas en vuestra casa teneis a fonseca del amor de dios donde se cifra todo lo que vos y el mas ingenioso acertare a desear en tal materia en resolucion no hay mas sino que vos procureis nombrar estos nombres o tocar estas historias en la vuestra que aqui he dicho y dejadme a mi el cargo de poner las anotaciones y acotaciones que yo os voto a tal de llenaros las margenes y de gastar cuatro pliegos en el fin del libro vengamos ahora a la citacion de los autores que los otros libros tienen que en el vuestro os faltan el remedio que esto tiene es muy facil porque no habeis de hacer otra cosa que buscar un libro que los acote todos desde la a hasta la z como vos decis pues ese mismo abecedario pondreis vos en vuestro libro que puesto que a la clara se vea la mentira por la poca necesidad que vos teniades de aprovecharos dellos no importa nada y quiza alguno habra tan simple que crea que de todos os habeis aprovechado en la simple y sencilla historia vuestra y cuando no sirva de otra cosa por lo menos servira aquel largo catalogo de autores a dar de improviso autoridad al libro y mas que no habra quien se ponga a averiguar si los seguistes o no los seguistes no yendole nada en ello cuanto mas que si bien caigo en la cuenta este vuestro libro no tiene necesidad de ninguna cosa de aquellas que vos decis que le falta porque todo el es una invectiva contra los libros de caballerias de quien nunca se acordo aristoteles ni dijo nada san basilio ni alcanzo ciceron ni caen debajo de la cuenta de sus fabulosos disparates las puntualidades de la verdad ni las observaciones de la astrologia ni le son de importancia las medidas geometricas ni la confutacion de los argumentos de quien se sirve la retorica ni tiene para que predicar a ninguno mezclando lo humano con lo divino que es un genero de mezcla de quien no se ha de vestir ningun cristiano entendimiento solo tiene que aprovecharse de la imitacion en lo que fuere escribiendo que cuanto ella fuere mas perfecta tanto mejor sera lo que se escribiere y pues esta vuestra escritura no mira a mas que a deshacer la autoridad y cabida que en el mundo y en el vulgo tienen los libros de caballerias no hay para que andeis mendigando sentencias de filosofos consejos de la divina escritura fabulas de poetas oraciones de retoricos milagros de santos sino procurar que a la llana con palabras significantes honestas y bien colocadas salga vuestra oracion y periodo sonoro y festivo pintando en todo lo que alcanzaredes y fuere posible vuestra intencion dando a entender vuestros conceptos sin intricarlos y escurecerlos procurad tambien que leyendo vuestra historia el melancolico se mueva a risa el risueno la acreciente el simple no se enfade el discreto se admire de la invencion el grave no la desprecie ni el prudente deje de alabarla en efecto llevad la mira puesta a derribar la maquina mal fundada destos caballerescos libros aborrecidos de tantos y alabados de muchos mas que si esto alcanzasedes no habriades alcanzado poco con silencio grande estuve escuchando lo que mi amigo me decia y de tal manera se imprimieron en mi sus razones que sin ponerlas en disputa las aprobe por buenas y de ellas mismas quise hacer este prologo en el cual veras lector suave la discrecion de mi amigo la buena ventura mia en hallar en tiempo tan necesitado tal consejero y el alivio tuyo en hallar tan sincera y tan sin revueltas la historia del famoso don quijote de la mancha de quien hay opinion por todos los habitadores del distrito del campo de montiel que fue el mas casto enamorado y el mas valiente caballero que de muchos anos a esta parte se vio en aquellos contornos yo no quiero encarecerte el servicio que te hago en darte a conocer tan noble y tan honrado caballero pero quiero que me agradezcas el conocimiento que tendras del famoso sancho panza su escudero en quien a mi parecer te doy cifradas todas las gracias escuderiles que en la caterva de los libros vanos de caballerias estan esparcidas y con esto dios te de salud y a mi no olvide vale al libro de don quijote de la mancha urganda la desconocida si de llegarte a los bue libro fueres con letu no te dira el boquirru que no pones bien los de mas si el pan no se te cue por ir a manos de idio veras de manos a bo aun no dar una en el cla si bien se comen las ma por mostrar que son curio y pues la expiriencia ense que el que a buen arbol se arri buena sombra le cobi en bejar tu buena estre un arbol real te ofre que da principes por fru en el cual florecio un du que es nuevo alejandro ma llega a su sombra que a osa favorece la fortu de un noble hidalgo manche contaras las aventu a quien ociosas letu trastornaron la cabe damas armas caballe le provocaron de mo que cual orlando furio templado a lo enamora alcanzo a fuerza de bra a dulcinea del tobo no indiscretos hierogli estampes en el escu que cuando es todo figu con ruines puntos se envi si en la direccion te humi no dira mofante algu que don alvaro de lu que anibal el de carta que rey francisco en espa se queja de la fortu pues al cielo no le plu que salieses tan ladi como el negro juan lati hablar latines rehu no me despuntes de agu ni me alegues con filo porque torciendo la bo dira el que entiende la le no un palmo de las ore para que conmigo flo no te metas en dibu ni en saber vidas aje que en lo que no va ni vie pasar de largo es cordu que suelen en caperu darles a los que grace mas tu quemate las ce solo en cobrar buena"

In [16]:
txt5 = "el ingenioso hidalgo don quijote de la mancha tasa yo juan gallo de andrada escribano de camara del rey nuestro senor de los que residen en su consejo certifico y doy fe que habiendo visto por los senores del un libro intitulado el ingenioso hidalgo de la mancha compuesto por miguel de cervantes saavedra tasaron cada pliego del dicho libro a tres maravedis y medio el cual tiene ochenta y tres pliegos que al dicho precio monta el dicho libro docientos y noventa maravedis y medio en que se ha de vender en papel y dieron licencia para que a este precio se pueda vender y mandaron que esta tasa se ponga al principio del dicho libro y no se pueda vender sin ella y para que dello conste di la presente en valladolid a veinte dias del mes de deciembre de mil y seiscientos y cuatro anos juan gallo de andrada testimonio de las erratas este libro no tiene cosa digna que no corresponda a su original en testimonio de lo haber correcto di esta fee en el colegio de la madre de dios de los teologos de la universidad de alcala en primero de diciembre de anos el licenciado francisco murcia de la llana el rey por cuanto por parte de vos miguel de cervantes nos fue fecha relacion que habiades compuesto un libro intitulado el ingenioso hidalgo de la mancha el cual os habia costado mucho trabajo y era muy util y provechoso nos pedistes y suplicastes os mandasemos dar licencia y facultad para le poder imprimir y previlegio por el tiempo que fuesemos servidos o como la nuestra merced fuese lo cual visto por los del nuestro consejo por cuanto en el dicho libro se hicieron las diligencias que la prematica ultimamente por nos fecha sobre la impresion de los libros dispone fue acordado que debiamos mandar dar esta nuestra cedula para vos en la dicha razon y nos tuvimoslo por bien por la cual por os hacer bien y merced os damos licencia y facultad para que vos o la persona que vuestro poder hubiere y no otra alguna podais imprimir el dicho libro intitulado el ingenioso hidalgo de la mancha que desuso se hace mencion en todos estos nuestros reinos de castilla por tiempo y espacio de diez anos que corran y se cuenten desde el dicho dia de la data desta nuestra cedula so pena que la persona o personas que sin tener vuestro poder lo imprimiere o vendiere o hiciere imprimir o vender por el mesmo caso pierda la impresion que hiciere con los moldes y aparejos della y mas incurra en pena de cincuenta mil maravedis cada vez que lo contrario hiciere la cual dicha pena sea la tercia parte para la persona que lo acusare y la otra tercia parte para nuestra camara y la otra tercia parte para el juez que lo sentenciare con tanto que todas las veces que hubieredes de hacer imprimir el dicho libro durante el tiempo de los dichos diez anos le traigais al nuestro consejo juntamente con el original que en el fue visto que va rubricado cada plana y firmado al fin del de juan gallo de andrada nuestro escribano de camara de los que en el residen para saber si la dicha impresion esta conforme el original o traigais fe en publica forma de como por corretor nombrado por nuestro mandado se vio y corrigio la dicha impresion por el original y se imprimio conforme a el y quedan impresas las erratas por el apuntadas para cada un libro de los que asi fueren impresos para que se tase el precio que por cada volume hubieredes de haber y mandamos al impresor que asi imprimiere el dicho libro no imprima el principio ni el primer pliego del ni entregue mas de un solo libro con el original al autor o persona a cuya costa lo imprimiere ni otro alguno para efeto de la dicha correcion y tasa hasta que antes y primero el dicho libro este corregido y tasado por los del nuestro consejo y estando hecho y no de otra manera pueda imprimir el dicho principio y primer pliego y sucesivamente ponga esta nuestra cedula y la aprobacion tasa y erratas so pena de caer e incurrir en las penas contenidas en las leyes y prematicas destos nuestros reinos y mandamos a los del nuestro consejo y a otras cualesquier justicias dellos guarden y cumplan esta nuestra cedula y lo en ella contenido fecha en valladolid a veinte y seis dias del mes de setiembre de mil y seiscientos y cuatro anos yo el rey por mandado del rey nuestro senor juan de amezqueta al duque de bejar marques de gibraleon conde de benalcazar y banares vizconde de la puebla de alcocer senor de las villas de capilla curiel y burguillos en fe del buen acogimiento y honra que hace vuestra excelencia a toda suerte de libros como principe tan inclinado a favorecer las buenas artes mayormente las que por su nobleza no se abaten al servicio y granjerias del vulgo he determinado de sacar a luz al ingenioso hidalgo don quijote de la mancha al abrigo del clarisimo nombre de vuestra excelencia a quien con el acatamiento que debo a tanta grandeza suplico le reciba agradablemente en su proteccion para que a su sombra aunque desnudo de aquel precioso ornamento de elegancia y erudicion de que suelen andar vestidas las obras que se componen en las casas de los hombres que saben ose parecer seguramente en el juicio de algunos que continiendose en los limites de su ignorancia suelen condenar con mas rigor y menos justicia los trabajos ajenos que poniendo los ojos la prudencia de vuestra excelencia en mi buen deseo fio que no desdenara la cortedad de tan humilde servicio miguel de cervantes saavedra prologo desocupado lector sin juramento me podras creer que quisiera que este libro como hijo del entendimiento fuera el mas hermoso el mas gallardo y mas discreto que pudiera imaginarse pero no he podido yo contravenir al orden de naturaleza que en ella cada cosa engendra su semejante y asi que podra engendrar el esteril y mal cultivado ingenio mio sino la historia de un hijo seco avellanado antojadizo y lleno de pensamientos varios y nunca imaginados de otro alguno bien como quien se engendro en una carcel donde toda incomodidad tiene su asiento y donde todo triste ruido hace su habitacion el sosiego el lugar apacible la amenidad de los campos la serenidad de los cielos el murmurar de las fuentes la quietud del espiritu son grande parte para que las musas mas esteriles se muestren fecundas y ofrezcan partos al mundo que le colmen de maravilla y de contento acontece tener un padre un hijo feo y sin gracia alguna y el amor que le tiene le pone una venda en los ojos para que no vea sus faltas antes las juzga por discreciones y lindezas y las cuenta a sus amigos por agudezas y donaires pero yo que aunque parezco padre soy padrastro de don quijote no quiero irme con la corriente del uso ni suplicarte casi con las lagrimas en los ojos como otros hacen lector carisimo que perdones o disimules las faltas que en este mi hijo vieres y ni eres su pariente ni su amigo y tienes tu alma en tu cuerpo y tu libre albedrio como el mas pintado y estas en tu casa donde eres senor della como el rey de sus alcabalas y sabes lo que comunmente se dice que debajo de mi manto al rey mato todo lo cual te esenta y hace libre de todo respecto y obligacion y asi puedes decir de la historia todo aquello que te pareciere sin temor que te calunien por el mal ni te premien por el bien que dijeres della solo quisiera dartela monda y desnuda sin el ornato de prologo ni de la inumerabilidad y catalogo de los acostumbrados sonetos epigramas y elogios que al principio de los libros suelen ponerse porque te se decir que aunque me costo algun trabajo componerla ninguno tuve por mayor que hacer esta prefacion que vas leyendo muchas veces tome la pluma para escribille y muchas la deje por no saber lo que escribiria y estando una suspenso con el papel delante la pluma en la oreja el codo en el bufete y la mano en la mejilla pensando lo que diria entro a deshora un amigo mio gracioso y bien entendido el cual viendome tan imaginativo me pregunto la causa y no encubriendosela yo le dije que pensaba en el prologo que habia de hacer a la historia de don quijote y que me tenia de suerte que ni queria hacerle ni menos sacar a luz las hazanas de tan noble caballero porque como quereis vos que no me tenga confuso el que dira el antiguo legislador que llaman vulgo cuando vea que al cabo de tantos anos como ha que duermo en el silencio del olvido salgo ahora con todos mis anos a cuestas con una leyenda seca como un esparto ajena de invencion menguada de estilo pobre de concetos y falta de toda erudicion y doctrina sin acotaciones en las margenes y sin anotaciones en el fin del libro como veo que estan otros libros aunque sean fabulosos y profanos tan llenos de sentencias de aristoteles de platon y de toda la caterva de filosofos que admiran a los leyentes y tienen a sus autores por hombres leidos eruditos y elocuentes pues que cuando citan la divina escritura no diran sino que son unos santos tomases y otros doctores de la iglesia guardando en esto un decoro tan ingenioso que en un renglon han pintado un enamorado destraido y en otro hacen un sermoncico cristiano que es un contento y un regalo oille o leelle de todo esto ha de carecer mi libro porque ni tengo que acotar en el margen ni que anotar en el fin ni menos se que autores sigo en el para ponerlos al principio como hacen todos por las letras del abc comenzando en aristoteles y acabando en xenofonte y en zoilo o zeuxis aunque fue maldiciente el uno y pintor el otro tambien ha de carecer mi libro de sonetos al principio a lo menos de sonetos cuyos autores sean duques marqueses condes obispos damas o poetas celeberrimos aunque si yo los pidiese a dos o tres oficiales amigos yo se que me los darian y tales que no les igualasen los de aquellos que tienen mas nombre en nuestra espana en fin senor y amigo mio prosegui yo determino que el senor don quijote se quede sepultado en sus archivos en la mancha hasta que el cielo depare quien le adorne de tantas cosas como le faltan porque yo me hallo incapaz de remediarlas por mi insuficiencia y pocas letras y porque naturalmente soy poltron y perezoso de andarme buscando autores que digan lo que yo me se decir sin ellos de aqui nace la suspension y elevamiento amigo en que me hallastes bastante causa para ponerme en ella la que de mi habeis oido oyendo lo cual mi amigo dandose una palmada en la frente y disparando en una carga de risa me dijo por dios hermano que agora me acabo de desenganar de un engano en que he estado todo el mucho tiempo que ha que os conozco en el cual siempre os he tenido por discreto y prudente en todas vuestras aciones pero agora veo que estais tan lejos de serlo como lo esta el cielo de la tierra como que es posible que cosas de tan poco momento y tan faciles de remediar puedan tener fuerzas de suspender y absortar un ingenio tan maduro como el vuestro y tan hecho a romper y atropellar por otras dificultades mayores a la fe esto no nace de falta de habilidad sino de sobra de pereza y penuria de discurso quereis ver si es verdad lo que digo pues estadme atento y vereis como en un abrir y cerrar de ojos confundo todas vuestras dificultades y remedio todas las faltas que decis que os suspenden y acobardan para dejar de sacar a la luz del mundo la historia de vuestro famoso don quijote luz y espejo de toda la caballeria andante decid le replique yo oyendo lo que me decia de que modo pensais llenar el vacio de mi temor y reducir a claridad el caos de mi confusion a lo cual el dijo lo primero en que reparais de los sonetos epigramas o elogios que os faltan para el principio y que sean de personajes graves y de titulo se puede remediar en que vos mesmo tomeis algun trabajo en hacerlos y despues los podeis bautizar y poner el nombre que quisieredes ahijandolos al preste juan de las indias o al emperador de trapisonda de quien yo se que hay noticia que fueron famosos poetas y cuando no lo hayan sido y hubiere algunos pedantes y bachilleres que por detras os muerdan y murmuren desta verdad no se os de dos maravedis porque ya que os averiguen la mentira no os han de cortar la mano con que lo escribistes en lo de citar en las margenes los libros y autores de donde sacaredes las sentencias y dichos que pusieredes en vuestra historia no hay mas sino hacer de manera que venga a pelo algunas sentencias o latines que vos sepais de memoria o a lo menos que os cuesten poco trabajo el buscalle como sera poner tratando de libertad y cautiverio non bene pro toto libertas venditur auro y luego en el margen citar a horacio o a quien lo dijo si trataredes del poder de la muerte acudir luego con pallida mors aequo pulsat pede pauperum tabernas regumque turres si de la amistad y amor que dios manda que se tenga al enemigo entraros luego al punto por la escritura divina que lo podeis hacer con tantico de curiosidad y decir las palabras por lo menos del mismo dios ego autem dico vobis diligite inimicos vestros si trataredes de malos pensamientos acudid con el evangelio de corde exeunt cogitationes malae si de la instabilidad de los amigos ahi esta caton que os dara su distico donec eris felix multos numerabis amicos tempora si fuerint nubila solus eris y con estos latinicos y otros tales os tendran siquiera por gramatico que el serlo no es de poca honra y provecho el dia de hoy en lo que toca el poner anotaciones al fin del libro seguramente lo podeis hacer desta manera si nombrais algun gigante en vuestro libro hacelde que sea el gigante golias y con solo esto que os costara casi nada teneis una grande anotacion pues podeis poner el gigante golias o goliat fue un filisteo a quien el pastor david mato de una gran pedrada en el valle de terebinto segun se cuenta en el libro de los reyes en el capitulo que vos hallaredes que se escribe tras esto para mostraros hombre erudito en letras humanas y cosmografo haced de modo como en vuestra historia se nombre el rio tajo y vereisos luego con otra famosa anotacion poniendo el rio tajo fue asi dicho por un rey de las espanas tiene su nacimiento en tal lugar y muere en el mar oceano besando los muros de la famosa ciudad de lisboa y es opinion que tiene las arenas de oro etc si trataredes de ladrones yo os dire la historia de caco que la se de coro si de mujeres rameras ahi esta el obispo de mondonedo que os prestara a lamia laida y flora cuya anotacion os dara gran credito si de crueles ovidio os entregara a medea si de encantadores y hechiceras homero tiene a calipso y virgilio a circe si de capitanes valerosos el mesmo julio cesar os prestara a si mismo en sus comentarios y plutarco os dara mil alejandros si trataredes de amores con dos onzas que sepais de la lengua toscana topareis con leon hebreo que os hincha las medidas y si no quereis andaros por tierras extranas en vuestra casa teneis a fonseca del amor de dios donde se cifra todo lo que vos y el mas ingenioso acertare a desear en tal materia en resolucion no hay mas sino que vos procureis nombrar estos nombres o tocar estas historias en la vuestra que aqui he dicho y dejadme a mi el cargo de poner las anotaciones y acotaciones que yo os voto a tal de llenaros las margenes y de gastar cuatro pliegos en el fin del libro vengamos ahora a la citacion de los autores que los otros libros tienen que en el vuestro os faltan el remedio que esto tiene es muy facil porque no habeis de hacer otra cosa que buscar un libro que los acote todos desde la a hasta la z como vos decis pues ese mismo abecedario pondreis vos en vuestro libro que puesto que a la clara se vea la mentira por la poca necesidad que vos teniades de aprovecharos dellos no importa nada y quiza alguno habra tan simple que crea que de todos os habeis aprovechado en la simple y sencilla historia vuestra y cuando no sirva de otra cosa por lo menos servira aquel largo catalogo de autores a dar de improviso autoridad al libro y mas que no habra quien se ponga a averiguar si los seguistes o no los seguistes no yendole nada en ello cuanto mas que si bien caigo en la cuenta este vuestro libro no tiene necesidad de ninguna cosa de aquellas que vos decis que le falta porque todo el es una invectiva contra los libros de caballerias de quien nunca se acordo aristoteles ni dijo nada san basilio ni alcanzo ciceron ni caen debajo de la cuenta de sus fabulosos disparates las puntualidades de la verdad ni las observaciones de la astrologia ni le son de importancia las medidas geometricas ni la confutacion de los argumentos de quien se sirve la retorica ni tiene para que predicar a ninguno mezclando lo humano con lo divino que es un genero de mezcla de quien no se ha de vestir ningun cristiano entendimiento solo tiene que aprovecharse de la imitacion en lo que fuere escribiendo que cuanto ella fuere mas perfecta tanto mejor sera lo que se escribiere y pues esta vuestra escritura no mira a mas que a deshacer la autoridad y cabida que en el mundo y en el vulgo tienen los libros de caballerias no hay para que andeis mendigando sentencias de filosofos consejos de la divina escritura fabulas de poetas oraciones de retoricos milagros de santos sino procurar que a la llana con palabras significantes honestas y bien colocadas salga vuestra oracion y periodo sonoro y festivo pintando en todo lo que alcanzaredes y fuere posible vuestra intencion dando a entender vuestros conceptos sin intricarlos y escurecerlos procurad tambien que leyendo vuestra historia el melancolico se mueva a risa el risueno la acreciente el simple no se enfade el discreto se admire de la invencion el grave no la desprecie ni el prudente deje de alabarla en efecto llevad la mira puesta a derribar la maquina mal fundada destos caballerescos libros aborrecidos de tantos y alabados de muchos mas que si esto alcanzasedes no habriades alcanzado poco con silencio grande estuve escuchando lo que mi amigo me decia y de tal manera se imprimieron en mi sus razones que sin ponerlas en disputa las aprobe por buenas y de ellas mismas quise hacer este prologo en el cual veras lector suave la discrecion de mi amigo la buena ventura mia en hallar en tiempo tan necesitado tal consejero y el alivio tuyo en hallar tan sincera y tan sin revueltas la historia del famoso don quijote de la mancha de quien hay opinion por todos los habitadores del distrito del campo de montiel que fue el mas casto enamorado y el mas valiente caballero que de muchos anos a esta parte se vio en aquellos contornos yo no quiero encarecerte el servicio que te hago en darte a conocer tan noble y tan honrado caballero pero quiero que me agradezcas el conocimiento que tendras del famoso sancho panza su escudero en quien a mi parecer te doy cifradas todas las gracias escuderiles que en la caterva de los libros vanos de caballerias estan esparcidas y con esto dios te de salud y a mi no olvide vale al libro de don quijote de la mancha urganda la desconocida si de llegarte a los bue libro fueres con letu no te dira el boquirru que no pones bien los de mas si el pan no se te cue por ir a manos de idio veras de manos a bo aun no dar una en el cla si bien se comen las ma por mostrar que son curio y pues la expiriencia ense que el que a buen arbol se arri buena sombra le cobi en bejar tu buena estre un arbol real te ofre que da principes por fru en el cual florecio un du que es nuevo alejandro ma llega a su sombra que a osa favorece la fortu de un noble hidalgo manche contaras las aventu a quien ociosas letu trastornaron la cabe damas armas caballe le provocaron de mo que cual orlando furio templado a lo enamora alcanzo a fuerza de bra a dulcinea del tobo no indiscretos hierogli estampes en el escu que cuando es todo figu con ruines puntos se envi si en la direccion te humi no dira mofante algu que don alvaro de lu que anibal el de carta que rey francisco en espa se queja de la fortu pues al cielo no le plu que salieses tan ladi como el negro juan lati hablar latines rehu no me despuntes de agu ni me alegues con filo porque torciendo la bo dira el que entiende la le no un palmo de las ore para que conmigo flo no te metas en dibu ni en saber vidas aje que en lo que no va ni vie pasar de largo es cordu que suelen en caperu darles a los que grace mas tu quemate las ce solo en cobrar buena fa que el que imprime neceda dalas a censo perpe advierte que es desati siendo de vidrio el teja tomar piedras en las ma para tirar al veci deja que el hombre de jui en las obras que compo se vaya con pies de plo que el que saca a luz pape para entretener donce escribe a tontas y a lo amadis de gaula a don quijote de la mancha soneto tu que imitaste la llorosa vida que tuve ausente y desdenado sobre el gran ribazo de la pena pobre de alegre a penitencia reducida tu a quien los ojos dieron la bebida de abundante licor aunque salobre y alzandote la plata estano y cobre te dio la tierra en tierra la comida vive seguro de que eternamente en tanto al menos que en la cuarta esfera sus caballos aguije el rubio apolo tendras claro renombre de valiente tu patria sera en todas la primera tu sabio autor al mundo unico y solo don belianis de grecia a don quijote de la mancha soneto rompi corte abolle y dije y hice mas que en el orbe caballero andante fui diestro fui valiente fui arrogante mil agravios vengue cien mil deshice hazanas di a la fama que eternice fui comedido y regalado amante fue enano para mi todo gigante y al duelo en cualquier punto satisfice tuve a mis pies postrada la fortuna y trajo del copete mi cordura a la calva ocasion al estricote mas aunque sobre el cuerno de la luna siempre se vio encumbrada mi ventura tus proezas envidio oh gran quijote la senora oriana a dulcinea del toboso soneto oh quien tuviera hermosa dulcinea por mas comodidad y mas reposo a miraflores puesto en el toboso y trocara sus londres con tu aldea oh quien de tus deseos y librea alma y cuerpo adornara y del famoso caballero que hiciste venturoso mirara alguna desigual pelea oh quien tan castamente se escapara del senor amadis como tu hiciste del comedido hidalgo don quijote que asi envidiada fuera y no envidiara y fuera alegre el tiempo que fue triste y gozara los gustos sin escote gandalin escudero de amadis de gaula a sancho panza escudero de don quijote soneto salve varon famoso a quien fortuna cuando en el trato escuderil te puso tan blanda y cuerdamente lo dispuso que lo pasaste sin desgracia alguna ya la azada o la hoz poco repugna al andante ejercicio ya esta en uso la llaneza escudera con que acuso al soberbio que intenta hollar la luna envidio a tu jumento y a tu nombre y a tus alforjas igualmente invidio que mostraron tu cuerda providencia salve otra vez oh sancho tan buen hombre que a solo tu nuestro espanol ovidio con buzcorona te hace reverencia del donoso poeta entreverado a sancho panza y rocinante soy sancho panza escude del manchego don quijo puse pies en polvoro por vivir a lo discre que el tacito villadie toda su razon de esta cifro en una retira segun siente celesti libro en mi opinion divi si encubriera mas lo huma a rocinante soy rocinante el famo bisnieto del gran babie por pecados de flaque fui a poder de un don quijo parejas corri a lo flo mas por una de caba no se me escapo ceba que esto saque a lazari cuando para hurtar el vi al ciego le di la pa orlando furioso a don quijote de la mancha soneto si no eres par tampoco le has tenido que par pudieras ser entre mil pares ni puede haberle donde tu te hallares invito vencedor jamas vencido orlando soy quijote que perdido por angelica vi remotos mares ofreciendo a la fama en sus altares aquel valor que respeto el olvido no puedo ser tu igual que este decoro se debe a tus proezas y a tu fama puesto que como yo perdiste el seso mas serlo has mio si al soberbio moro y cita fiero domas que hoy nos llama iguales en amor con mal suceso el caballero del febo a don quijote de la mancha soneto a vuestra espada no igualo la mia febo espanol curioso cortesano ni a la alta gloria de valor mi mano que rayo fue do nace y muere el dia imperios desprecie la monarquia que me ofrecio el oriente rojo en vano deje por ver el rostro soberano de claridiana aurora hermosa mia amela por milagro unico y raro y ausente en su desgracia el propio infierno temio mi brazo que domo su rabia mas vos godo quijote ilustre y claro por dulcinea sois al mundo eterno y ella por vos famosa honesta y sabia de solisdan a don quijote de la mancha soneto maguer senor quijote que sandeces vos tengan el cerbelo derrumbado nunca sereis de alguno reprochado por home de obras viles y soeces seran vuesas fazanas los joeces pues tuertos desfaciendo habeis andado siendo vegadas mil apaleado por follones cautivos y raheces y si la vuesa linda dulcinea desaguisado contra vos comete ni a vuesas cuitas muestra buen talante en tal desman vueso conorte sea que sancho panza fue mal alcaguete necio el dura ella y vos no amante dialogo entre babieca y rocinante soneto b como estais rocinante tan delgado r porque nunca se come y se trabaja b pues que es de la cebada y de la paja r no me deja mi amo ni un bocado b anda senor que estais muy mal criado pues vuestra lengua de asno al amo ultraja r asno se es de la cuna a la mortaja quereislo ver miraldo enamorado b es necedad amar r no es gran prudencia b metafisico estais r es que no como b quejaos del escudero r no es bastante como me he de quejar en mi dolencia si el amo y escudero o mayordomo son tan rocines como rocinante primera parte del ingenioso hidalgo don quijote de la mancha capitulo primero que trata de la condicion y ejercicio del famoso hidalgo don quijote de la mancha en un lugar de la mancha de cuyo nombre no quiero acordarme no ha mucho tiempo que vivia un hidalgo de los de lanza en astillero adarga antigua rocin flaco y galgo corredor una olla de algo mas vaca que carnero salpicon las mas noches duelos y quebrantos los sabados lantejas los viernes algun palomino de anadidura los domingos consumian las tres partes de su hacienda el resto della concluian sayo de velarte calzas de velludo para las fiestas con sus pantuflos de lo mesmo y los dias de entresemana se honraba con su vellori de lo mas fino tenia en su casa una ama que pasaba de los cuarenta y una sobrina que no llegaba a los veinte y un mozo de campo y plaza que asi ensillaba el rocin como tomaba la podadera frisaba la edad de nuestro hidalgo con los cincuenta anos era de complexion recia seco de carnes enjuto de rostro gran madrugador y amigo de la caza quieren decir que tenia el sobrenombre de quijada o quesada que en esto hay alguna diferencia en los autores que deste caso escriben aunque por conjeturas verosimiles se deja entender que se llamaba quejana pero esto importa poco a nuestro cuento basta que en la narracion del no se salga un punto de la verdad es pues de saber que este sobredicho hidalgo los ratos que estaba ocioso que eran los mas del ano se daba a leer libros de caballerias con tanta aficion y gusto que olvido casi de todo punto el ejercicio de la caza y aun la administracion de su hacienda y llego a tanto su curiosidad y desatino en esto que vendio muchas hanegas de tierra de sembradura para comprar libros de caballerias en que leer y asi llevo a su casa todos cuantos pudo haber dellos y de todos ningunos le parecian tan bien como los que compuso el famoso feliciano de silva porque la claridad de su prosa y aquellas entricadas razones suyas le parecian de perlas y mas cuando llegaba a leer aquellos requiebros y cartas de desafios donde en muchas partes hallaba escrito la razon de la sinrazon que a mi razon se hace de tal manera mi razon enflaquece que con razon me quejo de la vuestra fermosura y tambien cuando leia los altos cielos que de vuestra divinidad divinamente con las estrellas os fortifican y os hacen merecedora del merecimiento que merece la vuestra grandeza con estas razones perdia el pobre caballero el juicio y desvelabase por entenderlas y desentranarles el sentido que no se lo sacara ni las entendiera el mesmo aristoteles si resucitara para solo ello no estaba muy bien con las heridas que don belianis daba y recebia porque se imaginaba que por grandes maestros que le hubiesen curado no dejaria de tener el rostro y todo el cuerpo lleno de cicatrices y senales pero con todo alababa en su autor aquel acabar su libro con la promesa de aquella inacabable aventura y muchas veces le vino deseo de tomar la pluma y dalle fin al pie de la letra como alli se promete y sin duda alguna lo hiciera y aun saliera con ello si otros mayores y continuos pensamientos no se lo estorbaran tuvo muchas veces competencia con el cura de su lugar que era hombre docto graduado en siguenza sobre cual habia sido mejor caballero palmerin de ingalaterra o amadis de gaula mas maese nicolas barbero del mesmo pueblo decia que ninguno llegaba al caballero del febo y que si alguno se le podia comparar era don galaor hermano de amadis de gaula porque tenia muy acomodada condicion para todo que no era caballero melindroso ni tan lloron como su hermano y que en lo de la valentia no le iba en zaga en resolucion el se enfrasco tanto en su letura que se le pasaban las noches leyendo de claro en claro y los dias de turbio en turbio y asi del poco dormir y del mucho leer se le seco el celebro de manera que vino a perder el juicio llenosele la fantasia de todo aquello que leia en los libros asi de encantamentos como de pendencias batallas desafios heridas requiebros amores tormentas y disparates imposibles y asentosele de tal modo en la imaginacion que era verdad toda aquella maquina de aquellas sonadas sonadas invenciones que leia que para el no habia otra historia mas cierta en el mundo decia el que el cid ruy diaz habia sido muy buen caballero pero que no tenia que ver con el caballero de la ardiente espada que de solo un reves habia partido por medio dos fieros y descomunales gigantes mejor estaba con bernardo del carpio porque en roncesvalles habia muerto a roldan el encantado valiendose de la industria de hercules cuando ahogo a anteo el hijo de la tierra entre los brazos decia mucho bien del gigante morgante porque con ser de aquella generacion gigantea que todos son soberbios y descomedidos el solo era afable y bien criado pero sobre todos estaba bien con reinaldos de montalban y mas cuando le veia salir de su castillo y robar cuantos topaba y cuando en allende robo aquel idolo de mahoma que era todo de oro segun dice su historia diera el por dar una mano de coces al traidor de galalon al ama que tenia y aun a su sobrina de anadidura en efeto rematado ya su juicio vino a dar en el mas estrano pensamiento que jamas dio loco en el mundo y fue que le parecio convenible y necesario asi para el aumento de su honra como para el servicio de su republica hacerse caballero andante y irse por todo el mundo con sus armas y caballo a buscar las aventuras y a ejercitarse en todo aquello que el habia leido que los caballeros andantes se ejercitaban deshaciendo todo genero de agravio y poniendose en ocasiones y peligros donde acabandolos cobrase eterno nombre y fama imaginabase el pobre ya coronado por el valor de su brazo por lo menos del imperio de trapisonda y asi con estos tan agradables pensamientos llevado del estrano gusto que en ellos sentia se dio priesa a poner en efeto lo que deseaba y lo primero que hizo fue limpiar unas armas que habian sido de sus bisabuelos que tomadas de orin y llenas de moho luengos siglos habia que estaban puestas y olvidadas en un rincon limpiolas y aderezolas lo mejor que pudo pero vio que tenian una gran falta y era que no tenian celada de encaje sino morrion simple mas a esto suplio su industria porque de cartones hizo un modo de media celada que encajada con el morrion hacian una apariencia de celada entera es verdad que para probar si era fuerte y podia estar al riesgo de una cuchillada saco su espada y le dio dos golpes y con el primero y en un punto deshizo lo que habia hecho en una semana y no dejo de parecerle mal la facilidad con que la habia hecho pedazos y por asegurarse deste peligro la torno a hacer de nuevo poniendole unas barras de hierro por de dentro de tal manera que el quedo satisfecho de su fortaleza y sin querer hacer nueva experiencia della la diputo y tuvo por celada finisima de encaje fue luego a ver su rocin y aunque tenia mas cuartos que un real y mas tachas que el caballo de gonela que tantum pellis et ossa fuit le parecio que ni el bucefalo de alejandro ni babieca el del cid con el se igualaban cuatro dias se le pasaron en imaginar que nombre le pondria porque segun se decia el a si mesmo no era razon que caballo de caballero tan famoso y tan bueno el por si estuviese sin nombre conocido y ansi procuraba acomodarsele de manera que declarase quien habia sido antes que fuese de caballero andante y lo que era entonces pues estaba muy puesto en razon que mudando su senor estado mudase el tambien el nombre y le cobrase famoso y de estruendo como convenia a la nueva orden y al nuevo ejercicio que ya profesaba y asi despues de muchos nombres que formo borro y quito anadio deshizo y torno a hacer en su memoria e imaginacion al fin le vino a llamar rocinante nombre a su parecer alto sonoro y significativo de lo que habia sido cuando fue rocin antes de lo que ahora era que era antes y primero de todos los rocines del mundo puesto nombre y tan a su gusto a su caballo quiso ponersele a si mismo y en este pensamiento duro otros ocho dias y al cabo se vino a llamar don quijote de donde como queda dicho tomaron ocasion los autores desta tan verdadera historia que sin duda se debia de llamar quijada y no quesada como otros quisieron decir pero acordandose que el valeroso amadis no solo se habia contentado con llamarse amadis a secas sino que anadio el nombre de su reino y patria por hepila famosa y se llamo amadis de gaula asi quiso como buen caballero anadir al suyo el nombre de la suya y llamarse don quijote de la mancha con que a su parecer declaraba muy al vivo su linaje y patria y la honraba con tomar el sobrenombre della limpias pues sus armas hecho del morrion celada puesto nombre a su rocin y confirmandose a si mismo se dio a entender que no le faltaba otra cosa sino buscar una dama de quien enamorarse porque el caballero andante sin amores era arbol sin hojas y sin fruto y cuerpo sin alma deciase el a si si yo por malos de mis pecados o por mi buena suerte me encuentro por ahi con algun gigante como de ordinario les acontece a los caballeros andantes y le derribo de un encuentro o le parto por mitad del cuerpo o finalmente le venzo y le rindo no sera bien tener a quien enviarle presentado y que entre y se hinque de rodillas ante mi dulce senora y diga con voz humilde y rendido yo senora soy el gigante caraculiambro senor de la insula malindrania a quien vencio en singular batalla el jamas como se debe alabado caballero don quijote de la mancha el cual me mando que me presentase ante vuestra merced para que la vuestra grandeza disponga de mi a su talante oh como se holgo nuestro buen caballero cuando hubo hecho este discurso y mas cuando hallo a quien dar nombre de su dama y fue a lo que se cree que en un lugar cerca del suyo habia una moza labradora de muy buen parecer de quien el un tiempo anduvo enamorado aunque segun se entiende ella jamas lo supo ni le dio cata dello llamabase aldonza lorenzo y a esta le parecio ser bien darle titulo de senora de sus pensamientos y buscandole nombre que no desdijese mucho del suyo y que tirase y se encaminase al de princesa y gran senora vino a llamarla dulcinea del toboso porque era natural del toboso nombre a su parecer musico y peregrino y significativo como todos los demas que a el y a sus cosas habia puesto capitulo ii que trata de la primera salida que de su tierra hizo el ingenioso don quijote hechas pues estas prevenciones no quiso aguardar mas tiempo a poner en efeto su pensamiento apretandole a ello la falta que el pensaba que hacia en el mundo su tardanza segun eran los agravios que pensaba deshacer tuertos que enderezar sinrazones que emendar y abusos que mejorar y deudas que satisfacer y asi sin dar parte a persona alguna de su intencion y sin que nadie le viese una manana antes del dia que era uno de los calurosos del mes de julio se armo de todas sus armas subio sobre rocinante puesta su mal compuesta celada embrazo su adarga tomo su lanza y por la puerta falsa de un corral salio al campo con grandisimo contento y alborozo de ver con cuanta facilidad habia dado principio a su buen deseo mas apenas se vio en el campo cuando le asalto un pensamiento terrible y tal que por poco le hiciera dejar la comenzada empresa y fue que le vino a la memoria que no era armado caballero y que conforme a ley de caballeria ni podia ni debia tomar armas con ningun caballero y puesto que lo fuera habia de llevar armas blancas como novel caballero sin empresa en el escudo hasta que por su esfuerzo la ganase estos pensamientos le hicieron titubear en su proposito mas pudiendo mas su locura que otra razon alguna propuso de hacerse armar caballero del primero que topase a imitacion de otros muchos que asi lo hicieron segun el habia leido en los libros que tal le tenian en lo de las armas blancas pensaba limpiarlas de manera en teniendo lugar que lo fuesen mas que un armino y con esto se quieto y prosiguio su camino sin llevar otro que aquel que su caballo queria creyendo que en aquello consistia la fuerza de las aventuras yendo pues caminando nuestro flamante aventurero iba hablando consigo mesmo y diciendo quien duda sino que en los venideros tiempos cuando salga a luz la verdadera historia de mis famosos hechos que el sabio que los escribiere no ponga cuando llegue a contar esta mi primera salidad tan de manana desta manera apenas habia el rubicundo apolo tendido por la faz de la ancha y espaciosa tierra las doradas hebras de sus hermosos cabellos y apenas los pequenos y pintados pajarillos con sus arpadas lenguas habian saludado con dulce y meliflua armonia la venida de la rosada aurora que dejando la blanda cama del celoso marido por las puertas y balcones del manchego horizonte a los mortales se mostraba cuando el famoso caballero don quijote de la mancha dejando las ociosas plumas subio sobre su famoso caballo rocinante y comenzo a caminar por el antiguo y conocido campo de montiel y era la verdad que por el caminaba y anadio diciendo dichosa edad y siglo dichoso aquel adonde saldran a luz las famosas hazanas mias dignas de entallarse en bronces esculpirse en marmoles y pintarse en tablas para memoria en lo futuro oh tu sabio encantador quienquiera que seas a quien ha de tocar el ser coronista desta peregrina historia ruegote que no te olvides de mi buen rocinante companero eterno mio en todos mis caminos y carreras luego volvia diciendo como si verdaderamente fuera enamorado oh princesa dulcinea senora deste cautivo corazon mucho agravio me habedes fecho en despedirme y reprocharme con el riguroso afincamiento de mandarme no parecer ante la vuestra fermosura plegaos senora de membraros deste vuestro sujeto corazon que tantas cuitas por vuestro amor padece con estos iba ensartando otros disparates todos al modo de los que sus libros le habian ensenado imitando en cuanto podia su lenguaje con esto caminaba tan despacio y el sol entraba tan apriesa y con tanto ardor que fuera bastante a derretirle los sesos si algunos tuviera casi todo aquel dia camino sin acontecerle cosa que de contar fuese de lo cual se desesperaba porque quisiera topar luego luego con quien hacer experiencia del valor de su fuerte brazo autores hay que dicen que la primera aventura que le avino fue la del puerto lapice otros dicen que la de los molinos de viento pero lo que yo he podido averiguar en este caso y lo que he hallado escrito en los anales de la mancha es que el anduvo todo aquel dia y al anochecer su rocin y el se hallaron cansados y muertos de hambre y que mirando a todas partes por ver si descubriria algun castillo o alguna majada de pastores donde recogerse y adonde pudiese remediar su mucha hambre y necesidad vio no lejos del camino por donde iba una venta que fue como si viera una estrella que no a los portales sino a los alcazares de su redencion le encaminaba diose priesa a caminar y llego a ella a tiempo que anochecia estaban acaso a la puerta dos mujeres mozas destas que llaman del partido las cuales iban a sevilla con unos arrieros que en la venta aquella noche acertaron a hacer jornada y como a nuestro aventurero todo cuanto pensaba veia o imaginaba le parecia ser hecho y pasar al modo de lo que habia leido luego que vio la venta se le represento que era un castillo con sus cuatro torres y chapiteles de luciente plata sin faltarle su puente levadiza y honda cava con todos aquellos adherentes que semejantes castillos se pintan fuese llegando a la venta que a el le parecia castillo y a poco trecho della detuvo las riendas a rocinante esperando que algun enano se pusiese entre las almenas a dar senal con alguna trompeta de que llegaba caballero al castillo pero como vio que se tardaban y que rocinante se daba priesa por llegar a la caballeriza se llego a la puerta de la venta y vio a las dos destraidas mozas que alli estaban que a el le parecieron dos hermosas doncellas o dos graciosas damas que delante de la puerta del castillo se estaban solazando en esto sucedio acaso que un porquero que andaba recogiendo de unos rastrojos una manada de puercos que sin perdon asi se llaman toco un cuerno a cuya senal ellos se recogen y al instante se le represento a don quijote lo que deseaba que era que algun enano hacia senal de su venida y asi con estrano contento llego a la venta y a las damas las cuales como vieron venir un hombre de aquella suerte armado y con lanza y adarga llenas de miedo se iban a entrar en la venta pero don quijote coligiendo por su huida su miedo alzandose la visera de papelon y descubriendo su seco y polvoroso rostro con gentil talante y voz reposada les dijo no fuyan las vuestras mercedes ni teman desaguisado alguno ca a la orden de caballeria que profeso non toca ni atane facerle a ninguno cuanto mas a tan altas doncellas como vuestras presencias demuestran mirabanle las mozas y andaban con los ojos buscandole el rostro que la mala visera le encubria mas como se oyeron llamar doncellas cosa tan fuera de su profesion no pudieron tener la risa y fue de manera que don quijote vino a correrse y a decirles bien parece la mesura en las fermosas y es mucha sandez ademas la risa que de leve causa procede pero no vos lo digo porque os acuitedes ni mostredes mal talante que el mio non es de al que de serviros el lenguaje no entendido de las senoras y el mal talle de nuestro caballero acrecentaba en ellas la risa y en el el enojo y pasara muy adelante si a aquel punto no saliera el ventero hombre que por ser muy gordo era muy pacifico el cual viendo aquella figura contrahecha armada de armas tan desiguales como eran la brida lanza adarga y coselete no estuvo en nada en acompanar a las doncellas en las muestras de su contento mas en efeto temiendo la maquina de tantos pertrechos determino de hablarle comedidamente y asi le dijo si vuestra merced senor caballero busca posada amen del lecho porque en esta venta no hay ninguno todo lo demas se hallara en ella en mucha abundancia viendo don quijote la humildad del alcaide de la fortaleza que tal le parecio a el el ventero y la venta respondio para mi senor castellano cualquiera cosa basta porque mis arreos son las armas mi descanso el pelear etc penso el huesped que el haberle llamado castellano habia sido por haberle parecido de los sanos de castilla aunque el era andaluz y de los de la playa de sanlucar no menos ladron que caco ni menos maleante que estudiantado paje y asi le respondio segun eso las camas de vuestra merced seran duras penas y su dormir siempre velar y siendo asi bien se puede apear con seguridad de hallar en esta choza ocasion y ocasiones para no dormir en todo un ano cuanto mas en una noche y diciendo esto fue a tener el estribo a don quijote el cual se apeo con mucha dificultad y trabajo como aquel que en todo aquel dia no se habia desayunado dijo luego al huesped que le tuviese mucho cuidado de su caballo porque era la mejor pieza que comia pan en el mundo mirole el ventero y no le parecio tan bueno como don quijote decia ni aun la mitad y acomodandole en la caballeriza volvio a ver lo que su huesped mandaba al cual estaban desarmando las doncellas que ya se habian reconciliado con el las cuales aunque le habian quitado el peto y el espaldar jamas supieron ni pudieron desencajarle la gola ni quitalle la contrahecha celada que traia atada con unas cintas verdes y era menester cortarlas por no poderse quitar los nudos mas el no lo quiso consentir en ninguna manera y asi se quedo toda aquella noche con la celada puesta que era la mas graciosa y estrana figura que se pudiera pensar y al desarmarle como el se imaginaba que aquellas traidas y llevadas que le desarmaban eran algunas principales senoras y damas de aquel castillo les dijo con mucho donaire nunca fuera caballero de damas tan bien servido como fuera don quijote cuando de su aldea vino doncellas curaban del princesas del su rocino o rocinante que este es el nombre senoras mias de mi caballo y don quijote de la mancha el mio que puesto que no quisiera descubrirme fasta que las fazanas fechas en vuestro servicio y pro me descubrieran la fuerza de acomodar al proposito presente este romance viejo de lanzarote ha sido causa que sepais mi nombre antes de toda sazon pero tiempo vendra en que las vuestras senorias me manden y yo obedezca y el valor de mi brazo descubra el deseo que tengo de serviros las mozas que no estaban hechas a oir semejantes retoricas no respondian palabra solo le preguntaron si queria comer alguna cosa cualquiera yantaria yo respondio don quijote porque a lo que entiendo me haria mucho al caso a dicha acerto a ser viernes aquel dia y no habia en toda la venta sino unas raciones de un pescado que en castilla llaman abadejo y en andalucia bacallao y en otras partes curadillo y en otras truchuela preguntaronle si por ventura comeria su merced truchuela que no habia otro pescado que dalle a comer como haya muchas truchuelas respondio don quijote podran servir de una trucha porque eso se me da que me den ocho reales en sencillos que en una pieza de a ocho cuanto mas que podria ser que fuesen estas truchuelas como la ternera que es mejor que la vaca y el cabrito que el cabron pero sea lo que fuere venga luego que el trabajo y peso de las armas no se puede llevar sin el gobierno de las tripas pusieronle la mesa a la puerta de la venta por el fresco y trujole el huesped una porcion del mal remojado y peor cocido bacallao y un pan tan negro y mugriento como sus armas pero era materia de grande risa verle comer porque como tenia puesta la celada y alzada la visera no podia poner nada en la boca con sus manos si otro no se lo daba y ponia y ansi una de aquellas senoras servia deste menester mas al darle de beber no fue posible ni lo fuera si el ventero no horadara una cana y puesto el un cabo en la boca por el otro le iba echando el vino y todo esto lo recebia en paciencia a trueco de no romper las cintas de la celada estando en esto llego acaso a la venta un castrador de puercos y asi como llego sono su silbato de canas cuatro o cinco veces con lo cual acabo de confirmar don quijote que estaba en algun famoso castillo y que le servian con musica y que el abadejo eran truchas el pan candeal y las rameras damas y el ventero castellano del castillo y con esto daba por bien empleada su determinacion y salida mas lo que mas le fatigaba era el no verse armado caballero por parecerle que no se podria poner legitimamente en aventura alguna sin recebir la orden de caballeria capitulo iii donde se cuenta la graciosa manera que tuvo don quijote en armarse caballero y asi fatigado deste pensamiento abrevio su venteril y limitada cena la cual acabada llamo al ventero y encerrandose con el en la caballeriza se hinco de rodillas ante el diciendole no me levantare jamas de donde estoy valeroso caballero fasta que la vuestra cortesia me otorgue un don que pedirle quiero el cual redundara en alabanza vuestra y en pro del genero humano el ventero que vio a su huesped a sus pies y oyo semejantes razones estaba confuso mirandole sin saber que hacerse ni decirle y porfiaba con el que se levantase y jamas quiso hasta que le hubo de decir que el le otorgaba el don que le pedia no esperaba yo menos de la gran magnificencia vuestra senor mio respondio don quijote y asi os digo que el don que os he pedido y de vuestra liberalidad me ha sido otorgado es que manana en aquel dia me habeis de armar caballero y esta noche en la capilla deste vuestro castillo velare las armas y manana como tengo dicho se cumplira lo que tanto deseo para poder como se debe ir por todas las cuatro partes del mundo buscando las aventuras en pro de los menesterosos como esta a cargo de la caballeria y de los caballeros andantes como yo soy cuyo deseo a semejantes fazanas es inclinado el ventero que como esta dicho era un poco socarron y ya tenia algunos barruntos de la falta de juicio de su huesped acabo de creerlo cuando acabo de oirle semejantes razones y por tener que reir aquella noche determino de seguirle el humor y asi le dijo que andaba muy acertado en lo que deseaba y pedia y que tal prosupuesto era propio y natural de los caballeros tan principales como el parecia y como su gallarda presencia mostraba y que el ansimesmo en los anos de su mocedad se habia dado a aquel honroso ejercicio andando por diversas partes del mundo buscando sus aventuras sin que hubiese dejado los percheles de malaga islas de riaran compas de sevilla azoguejo de segovia la olivera de valencia rondilla de granada playa de sanlucar potro de cordoba y las ventillas de toledo y otras diversas partes donde habia ejercitado la ligereza de sus pies sutileza de sus manos haciendo muchos tuertos recuestando muchas viudas deshaciendo algunas doncellas y enganando a algunos pupilos y finalmente dandose a conocer por cuantas audiencias y tribunales hay casi en toda espana y que a lo ultimo se habia venido a recoger a aquel su castillo donde vivia con su hacienda y con las ajenas recogiendo en"

In [3]:
quijote = open("quijote_clean.txt","r",encoding="utf-8").read()
quijote = quijote[:500000]

In [4]:
len(quijote)

500000

In [5]:
txt = quijote

### Code

In [48]:
alp = sorted(list(set(txt)))

In [49]:
bw = encode_burrows_wheeler(txt)
bw_code = bw[0]
index = bw[1]

In [50]:
mtf_code = encode_move_to_front(bw_code)

In [51]:
max_digits = len(str(max(mtf_code)))

mtf_code = [str(x).zfill(max_digits) for x in mtf_code]
mtf_code = ''.join(mtf_code)

In [52]:
src = source_fromtext(mtf_code,2)
huf = huffman_code(mtf_code, src, 2)
corr = dict(huf)
huf_code = encode(mtf_code,corr)

In [53]:
coded_txt_bytes = int(huf_code, 2).to_bytes((len(huf_code) + 7) // 8, byteorder='big')

In [54]:
coded_txt = coded_txt_bytes
#print(coded_txt)

### Decode

In [ ]:
huf = huffman_code(huf_code,src,2)
corr = dict(huf)
huf_decoded = decode(huf_code,corr)

In [ ]:
huf_decoded = [int((huf_decoded[i:i+max_digits])) for i in range(0, len(huf_decoded), max_digits)]

In [ ]:
mtf_decoded = decode_move_to_front(huf_decoded,alp)

In [ ]:
txt_decoded = decode_burrows_wheeler(mtf_decoded,index)

In [ ]:
#txt_decoded

### Performance

In [55]:
unicode_chars = len(txt)
coded_bytes = len(coded_txt)
performance = (coded_bytes / unicode_chars) * 8
print('performance:',performance)

performance: 3.37584
